In [2]:
import itertools
import zpu as zpu
import re
import sys
import time

In [3]:
def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print('%s function took %0.3f ms' % (f.__name__, (time2-time1)*1000.0))
        return(ret)
    return wrap

In [4]:
def subs(dic, n):
    s = dic["0"]
    i = 1
    while len(s) < n:
        s = s + dic[s[i]]
        i = i + 1
    return s[:n]

In [5]:
cpattern = re.compile("^R+$") # sama R
dpattern = re.compile("^(01)+$") # slovo 0101010101
epattern = re.compile("((0R){20})|((0E){20})|((1R){20})|((1E){20})$")
fpattern = re.compile("((((0R)+1E){10})|(((0E)+1R){10})|(((1R)+0E){10})|(((1E)+0R){10}))$")

In [6]:
import logging

log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)

In [7]:
log.debug("asdsa")
log.info("asdsa")

DEBUG:__main__:asdsa
INFO:__main__:asdsa


In [8]:
from multiprocessing import Pool
l = 300

def testwords(rule):
    dicti = {"1": rule[1], "0": rule[0]}
    word = subs(dicti, l)
    ret = zpu.isZps(word)
    if (ret[0]==True):
        bis = [ret[1], ret[2]]
        bis_c = zpu.makeBiseq(ret[1], ret[2])
        bisR = zpu.maximizeRinBiseq(ret[1], ret[2])
        if not (re.match(cpattern, bisR[1])) and \
        not (re.match(dpattern, word))and \
        not (re.search(epattern, bis_c)) and \
        not (re.search(fpattern, bis_c)):
            return  str(dicti) + " "+ " slovo: " + word[:40] + " " + str(bis)
def getresults(lphi0, lphi1):
    pool = Pool(processes=4)
    results = []
    for rep1 in range(1,lphi0):
        for rep2 in range(1,lphi1+1):
            phi0 = ['0'+ ''.join(i) for i in itertools.product('01', repeat=rep1)]
            phi1 = [''.join(j) for j in itertools.product('01', repeat=rep2)]
            cart = [ k for k in itertools.product(phi0, phi1)]
            r = pool.map_async(testwords, cart)
            results.append(list(filter(lambda x: x != None, r.get())))
    return results

def testwords2(rule):
    dicti = {"1": rule[1], "0": rule[0]}
    word = subs(dicti, l)
    ret = zpu.isZps2(word)
    if (ret[0]==True):
        bis = [ret[1], ret[2]]
        bis_c = zpu.makeBiseq(ret[1], ret[2])
        bisR = zpu.maximizeRinBiseq(ret[1], ret[2])
        if not (re.match(cpattern, bisR[1])) and \
        not (re.match(dpattern, word))and \
        not (re.search(epattern, bis_c)) and \
        not (re.search(fpattern, bis_c)):
            return  str(dicti) + " "+ " slovo: " + word[:40] + " " + str(bis)
def getresults2(lphi0, lphi1):
    pool = Pool(processes=4)
    results = []
    for rep1 in range(1,lphi0):
        for rep2 in range(1,lphi1+1):
            phi0 = ['0'+ ''.join(i) for i in itertools.product('01', repeat=rep1)]
            phi1 = [''.join(j) for j in itertools.product('01', repeat=rep2)]
            cart = [ k for k in itertools.product(phi0, phi1)]
            r = pool.map_async(testwords, cart)
            results.append(list(filter(lambda x: x != None, r.get())))
    return results

In [9]:
%time results = getresults(5, 5)
%time results2 = getresults2(5, 5)

for result in results:
    for words in result:
        print(words)
print("")
for result in results2:
    for words in result:
        print(words)

CPU times: user 63.3 ms, sys: 23.3 ms, total: 86.7 ms
Wall time: 14.9 s
CPU times: user 60 ms, sys: 16.7 ms, total: 76.7 ms
Wall time: 15 s
{'1': '10', '0': '01'}  slovo: 0110100110010110100101100110100110010110 ['011111111', 'RERERERER']
{'1': '1001', '0': '0110'}  slovo: 0110100110010110100101100110100110010110 ['011111111', 'RERERERER']

{'1': '10', '0': '01'}  slovo: 0110100110010110100101100110100110010110 ['011111111', 'RERERERER']
{'1': '1001', '0': '0110'}  slovo: 0110100110010110100101100110100110010110 ['011111111', 'RERERERER']


In [29]:
totest = (("01110", "10001"),("011110", "100001"), ("001110", "0011100011"),
              ("000111", "00000010111111"), ("011001", "10011001100110"),
              ("0101110", "0010110"), ("0010111", "0111010"), ("0011010","0111010"),
             ("0011010","1001110"))
%time r = map(testwords, totest)
print(list(r))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 18.1 µs
[None, None, None, None, "{'1': '10011001100110', '0': '011001'}  slovo: 0110011001100110011010011001100110011001 ['01101010101101010101', 'RERERERERERERERERERE']", None, None, None, None]


In [9]:
zpu.isZps2("01001010010")

[True, '01001', 'RERRR']

In [ ]:
dicti = {"1": "10001", "0": "01110"}
w = subs(dicti, 200)
w2 = zpu.makeWord("011111111111", "RERERERERERE", 9)
print(w)
print(w2)
print(w[:100] == w2[:100])

In [ ]:
def tests():
    results = []
    for rep1 in range(1,5):
        for rep2 in range(1,5):
            phi0 = ['0'+ ''.join(i) for i in itertools.product('01', repeat=rep1)]
            phi1 = [''.join(j) for j in itertools.product('01', repeat=rep2)]
            cart = [ k for k in itertools.product(phi0, phi1)]
            n = 500
            for rule in cart:
                dicti = {"1": rule[1], "0": rule[0]}
                word = subs(dicti, n)
                ret = zpu.isZps(word)
                if (ret[0]==True):
                    bis = [ret[1], ret[2]]
                    bis_c = zpu.makeBiseq(ret[1], ret[2])
                    bisR = zpu.maximizeRinBiseq(ret[1], ret[2])
                    if not (re.match(cpattern, bisR[1])) and \
                    not (re.match(dpattern, word))and \
                    not (re.search(epattern, bis_c)) and \
                    not (re.search(fpattern, bis_c)):
                        results.append( str(dicti) + " "+ \
                                       " slovo: " + word[:40] + " " + str(bis))
                        
    return results

%time ress = tests()

for res in ress:
    print(res)

In [ ]:
def test(problematic, n):
    print(len(problematic))
    vyhozene = []
    zustavajici = []
    j = 0
    bs = []
    for rule in problematic:
                dicti = {"0": rule[0], "1": rule[1]}
                word = subs(dicti, n)
                ret = zpu.isZps(word)
                if (ret[0]==True):
                    bis = zpu.maximizeRinBiseq(ret[1], ret[2])
                    if not (re.match(cpattern, bis[1])) and \
                    not (re.match(dpattern, word)):
                        j = j + 1
                        print(str(dicti) + str(ret[0]) + " slovo: " + word[:100])
                        print(bis)
                        print("")
                    zustavajici.append((dicti["0"], dicti["1"]))
                    bs.append(bis)
                else:
                    s = str(dicti) + str(ret[0]) + " slovo: " + word[:50]
                    vyhozene.append([s,ret])
    print("")
    print("Vyhozene:")
    for vyhozena in vyhozene:
        print(vyhozena)
        print("")
    return (zustavajici, bs)

In [ ]:
'''problematic = (("01", "10"),("0110110", "1001001"), ("0110110", "110110110"),
              ("01010101", "01010101011"), ("0101010101", "1010101011"),
              ("010101010", "01010101011"), ("010101010", "10101010011"),
              ("010101010", "10101010110"), ("0110110110", "110110"),
              ("0101010101", "010101011"), ("0110110110", "110110110"),
              ("0101010101", "0101010011"), ("0101010101", "0101010110"),
              ("0101010101", "0101011001"), ("0110011001", "1001100110"),
              ("0110110110", "1001001001"), ("0101010101", "01010101011"),
              ("01010101010", "10101011"), ("01010101010", "101010011"),
              ("01010101010", "101010110"), ("01010101010", "101011001"),
              ("01010101010", "1010101011"), ("010101010", "01010101011"),
              ("010101010", "10101010011"), ("010101010", "10101010110"),
              ("0110110110", "110110"), ("0101010101", "010101011"),
              ("0110110110", "110110110"), ("01010101010", "10101010011"), 
              ("01010101010","10101010110"), ("01010101010", "10101011001"))'''
n = 500
vysledekcelkovy = test(problematic, n)
vysledek3 = vysledekcelkovy[0]

In [ ]:
vysledek = vysledek3
vysledek.append(("0110110110110", "1001001001001"))
vysledek.append(("0110", "1001"))
vysledek.append(("01101001", "10010110"))
r = test(vysledek, 10000)
l=list(zip(r[0],r[1]))
print("")
print("")
for el in l:
    print(el)